In [ ]:
import pandas as pd
import geopandas

In [ ]:
poi = geopandas.read_file("data/points-of-interest.geojson")
samm = geopandas.read_file("data/sammelstellen.geojson")
robidog = geopandas.read_file("data/hundekotbehalter.geojson")
mobility = geopandas.read_file('data/mobility-stationen-und-fahrzeuge-schweiz.geojson')
begegnung = geopandas.read_file('data/begegnungszonen.geojson')
tempo30 = geopandas.read_file('data/tempo-30-zonen.geojson')
veloplaetze = geopandas.read_file('data/veloabstellplatze.geojson')
parkplaetze = geopandas.read_file('data/parkplatze-und-parkhauser-stadt-stgallen.geojson')
beleuchtung = geopandas.read_file('data/offentliche-beleuchtung-stadt-stgallen.geojson')
eladestation = geopandas.read_file('data/ladestationen-fur-elektroautos-im-kanton-stgallen.geojson')
ovhaltestelle = geopandas.read_file('data/haltestelle-didok.geojson')
eventplaetze = geopandas.read_file('data/eventplatze-offentlicher-raum-stadt-stgallen.geojson')

In [ ]:
datasets = {
    'Sammelstelle': samm,
    'Robidog': robidog,
    # 'Mobility': mobility,
    'Begegnung': begegnung,
    'Tempo30': tempo30,
    'VeloPlaetze': veloplaetze,
    'Parkplaetze': parkplaetze,
    # 'Beleuchtung': beleuchtung,
    # 'E-Ladestation': eladestation,
    'OV-Haltestelle': ovhaltestelle,
    'EventPlaetze': eventplaetze
}

In [ ]:
merged = geopandas.read_file('data/merged.csv')
data = geopandas.GeoDataFrame(merged, geometry=geopandas.points_from_xy(merged.longitude, merged.latitude))
data = data.set_crs('EPSG:4326', allow_override=True) # coordinates
print('CRS added:',data.crs is not None)
print('Columns before:', len(data.columns))
print('Rows before:',len(data.index))

In [ ]:
def join_nearest(left_set, right_set, title, max_dist=1000, plot=True):
    col_name = f'dist_to_{title.lower()}'
    if col_name in left_set.columns:
        print(f'{title} exists already, skipping...')
        return left_set
    else:
        left_meters = left_set.to_crs('EPSG:3857') # meters
        right_meters = right_set.to_crs('EPSG:3857') # meters
        res = left_meters.sjoin_nearest(right=right_meters, how='left', max_distance=max_dist, distance_col=col_name, lsuffix=title.lower())
        res = res.loc[:, res.columns == col_name]
        left_set[col_name] = res
        if plot:
            res.plot(col_name, legend=True)
        print(f'{title} added')
        return left_set

In [ ]:
for x, d in enumerate(datasets):
    print('dataset:', f'{x+1}/{len(datasets)}')
    data = join_nearest(data, datasets[d], d, max_dist=10000, plot=False)
print('Columns after:', len(data.columns))
print('Rows after:',len(data.index))

In [ ]:
data


In [ ]:
data.to_csv('merged_new_cols.csv')